# Predict household income from satellite imagery data

First pass.

General ML pipeline steps:
1. Import data
2. Split data into test/train sets
3. Preprocess test/train sets separately
4. Generate features from data
5. For each regressor-hyperparameter combination:
    - Train regressor with given hyperparameters and training data and labels
    - Generate predicted labels for test data with trained regressor
    - Evaluate regressor-hyperparameter performance against actual test labels and get $R^2$
6. Explore best-performing models

In [17]:
import os
import math
import pickle
import numpy as np
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import LinearSVR
from sklearn.svm import LinearSVC

from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier, RandomForestClassifier

from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt

# Import configuration file
import config as cf

# Display options 
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = -1

# Turn off big pink warnings
import warnings
warnings.filterwarnings('ignore')

# Data file path 
final_data_file_path = "/Users/robmarty/Dropbox/World Bank/IEs/Pakistan Poverty Estimation from Satellites/Data/FinalData"
#


In [18]:
# Test grid to make sure everything works - limited models and parameters
GRID_TEST_CLASS = {
    'regressors': ['LinearSVC','DecisionTreeClassifier', 'BaggingClassifier',
                    'GradientBoostingClassifier', 'RandomForestClassifier'],
    'LinearSVC': [
        {'penalty': penalty, 'C': C, 'loss': loss, 'max_iter': max_iter,
        'random_state': 0} \
        for penalty in ('l2', ) \
        for C in (1e-2, 1e2) \
        for loss in ('epsilon_insensitive', ) \
        for max_iter in (1e3, 1e5)
    ],
    'DecisionTreeClassifier': [
        {'criterion': criterion, 'splitter': splitter, 'max_depth': max_depth,
        'max_features': max_features, 'random_state': 0} \
        for criterion in ('gini', ) \
        for splitter in ('best', ) \
        for max_depth in (1, 5, 10, 20, 30) \
        for max_features in ('sqrt', ) \
    ],
    'BaggingClassifier': [
        {'n_estimators': n_estimators, 'max_features': max_features,
        'random_state': 0, 'n_jobs': -1} \
        for n_estimators in (100, 1000, 10000) \
        for max_features in (0.3, 0.5, 1.0)
    ],
    'GradientBoostingClassifier': [
        {'loss': loss, 'learning_rate': rate, 'n_estimators': n_estimators,
        'criterion': criterion, 'max_features': max_features,
        'random_state': 0} \
        for loss in ('deviance', ) \
        for rate in (1e-4, )
        for n_estimators in (100, ) \
        for criterion in ('friedman_mse', ) \
        for max_features in ('sqrt', ) \
    ],
    'RandomForestClassifier': [
        {'n_estimators': n_estimators, 'criterion': criterion,
        'max_depth': max_depth, 'max_features': max_features, 'n_jobs': -1,
        'random_state': 0} \
        for n_estimators in (10, 100, 1000) \
        for criterion in ('gini', ) \
        for max_depth in (1, ) \
        for max_features in ('sqrt', )
    ]
}

## 1. Import data and drop "future" rows

In [19]:
DATA_PATH = os.path.join(final_data_file_path, 'Outputs for Analysis TEMP', 'bisp_sat_pov_data.csv')
df = pd.read_csv(DATA_PATH)
df.shape

(5416, 46)

In [20]:
df.head()

,uid,viirs_2012,viirs_2013,viirs_2014,viirs_2015,viirs_2016,viirs_2017,viirs_2018,dmspols_1992,dmspols_1993,dmspols_1994,dmspols_1995,dmspols_1996,dmspols_1997,dmspols_1998,dmspols_1999,dmspols_2000,dmspols_2001,dmspols_2002,dmspols_2003,dmspols_2004,dmspols_2005,dmspols_2006,dmspols_2007,dmspols_2008,dmspols_2009,dmspols_2010,dmspols_2011,dmspols_2012,dmspols_2013,l7_2011_1,l7_2011_2,l7_2011_3,l7_2011_4,l7_2011_5,l7_2011_6,l7_2011_7,l7_2013_1,l7_2013_2,l7_2013_3,l7_2013_4,l7_2013_5,l7_2013_6,l7_2013_7,pscores_2011,pscores_2013
0,100389,2.052018,2.141392,2.089507,2.307763,2.850603,3.653005,3.750000,43.0,33.666667,35.50,45.333333,40.0,33.166667,39.50,40.333333,37.333333,39.666667,38.833333,33.666667,34.00,34.50,40.666667,45.00,43.0,30.333333,46.0,32.666667,47.666667,45.333333,902.331348,1224.739396,1393.123911,2555.792708,2474.174317,3005.856769,1922.539802,951.897734,1282.748257,1417.251598,2574.000436,2469.137711,3000.408919,1889.609384,23.370001,29.889999
1,100401,1.964332,2.133366,2.052437,2.296554,2.769960,3.702374,3.488333,43.0,33.666667,35.50,45.333333,40.0,33.166667,39.50,40.333333,37.333333,39.666667,38.833333,33.666667,34.00,34.50,40.666667,45.00,43.0,30.333333,46.0,32.666667,47.666667,45.333333,885.841488,1200.548350,1366.253764,2512.672843,2451.849595,3004.616242,1890.566155,941.063694,1268.392009,1402.777070,2547.212362,2463.117111,2998.701940,1876.871453,5.110000,2.650000
2,100581,1.824753,1.937131,1.875487,2.047540,2.557241,3.198625,3.286000,43.0,32.500000,34.25,43.000000,38.0,31.750000,38.25,38.750000,36.000000,38.250000,37.750000,32.000000,32.75,33.75,40.000000,43.75,42.5,30.000000,45.5,30.500000,47.500000,44.500000,886.021385,1206.745127,1373.031277,2550.999418,2462.909660,3006.164678,1900.649840,935.162060,1263.157696,1398.079866,2572.847832,2458.750073,2999.056008,1880.909223,4.300000,5.510000
3,101101,1.964332,2.133366,2.052437,2.296554,2.769960,3.702374,3.488333,43.0,33.666667,35.50,45.333333,40.0,33.166667,39.50,40.333333,37.333333,39.666667,38.833333,33.666667,34.00,34.50,40.666667,45.00,43.0,30.333333,46.0,32.666667,47.666667,45.333333,886.196798,1201.037263,1366.468559,2514.479913,2450.865939,3004.699563,1890.108734,940.979913,1268.248763,1401.871616,2547.740466,2459.946143,2998.786463,1874.074672,8.190000,22.400000
4,101236,2.052018,2.141392,2.089507,2.307763,2.850603,3.653005,3.750000,43.0,33.666667,35.50,45.333333,40.0,33.166667,39.50,40.333333,37.333333,39.666667,38.833333,33.666667,34.00,34.50,40.666667,45.00,43.0,30.333333,46.0,32.666667,47.666667,45.333333,891.264553,1209.613090,1374.709528,2535.919345,2453.881552,3005.134086,1897.493484,943.113959,1271.824645,1403.386186,2563.689111,2453.279467,2999.725311,1869.047929,40.750000,NaN


In [21]:
# Remove NAs
df = df.loc[(df['pscores_2011'] >= 0)]
df = df.loc[(df['pscores_2011'] <= 20000000)]

In [22]:
# DV as Quantiles
#df['pscores_2011'] = pd.qcut(df['pscores_2011'], 3, labels=False)
#df['pscores_2011'].value_counts()
df['hhinc_2011'] = df['pscores_2011'] <= 16.7

In [24]:
df = df.drop(['pscores_2011'], axis=1)

In [25]:
# Keep only 2011 columns, but include viirs_2012
df = df.filter(regex='_2011', axis=1).join(df[['viirs_2012','uid']])
df.head()

,dmspols_2011,l7_2011_1,l7_2011_2,l7_2011_3,l7_2011_4,l7_2011_5,l7_2011_6,l7_2011_7,hhinc_2011,viirs_2012,uid
0,32.666667,902.331348,1224.739396,1393.123911,2555.792708,2474.174317,3005.856769,1922.539802,False,2.052018,100389
1,32.666667,885.841488,1200.548350,1366.253764,2512.672843,2451.849595,3004.616242,1890.566155,True,1.964332,100401
2,30.500000,886.021385,1206.745127,1373.031277,2550.999418,2462.909660,3006.164678,1900.649840,True,1.824753,100581
3,32.666667,886.196798,1201.037263,1366.468559,2514.479913,2450.865939,3004.699563,1890.108734,True,1.964332,101101
4,32.666667,891.264553,1209.613090,1374.709528,2535.919345,2453.881552,3005.134086,1897.493484,False,2.052018,101236


In [26]:
# Drop columns where the label is missing
df = df.loc[~pd.isnull(df['hhinc_2011'])]

df.shape

(4829, 11)

## 2. Split data into test/train

In [27]:
LABEL = 'hhinc_2011'
TEST_SIZE = 0.3

# Separate feature sets from label sets
x_df = df.drop(labels=[LABEL], axis=1)
y_df = df[LABEL]

# Split into test and train sets for features and labels
x_train, x_test, y_train, y_test =  train_test_split(x_df, y_df, test_size=TEST_SIZE)

## 3. Preprocess data

All vars are numeric - impute missing data with mean

In [28]:
# Check how many rows are missing across columns
print("TRAINING FEATURES MISSING:")
print(pd.isnull(x_train).sum())
print("")
print("TEST FEATURES MISSING:")
print(pd.isnull(x_test).sum())

TRAINING FEATURES MISSING:
dmspols_2011    32
l7_2011_1       0 
l7_2011_2       0 
l7_2011_3       0 
l7_2011_4       0 
l7_2011_5       0 
l7_2011_6       0 
l7_2011_7       0 
viirs_2012      32
uid             0 
dtype: int64

TEST FEATURES MISSING:
dmspols_2011    20
l7_2011_1       0 
l7_2011_2       0 
l7_2011_3       0 
l7_2011_4       0 
l7_2011_5       0 
l7_2011_6       0 
l7_2011_7       0 
viirs_2012      20
uid             0 
dtype: int64


In [29]:
for i in (x_train, x_test):
    for j in i.columns:
        
        if i[j].isnull().sum():
            # Create imputed flag
            new_name = i[j].name + '_imputed'
            i[new_name] = pd.isnull(i[j]).astype('int')
            # Fill with mean
            i[j] = i[j].fillna(i[j].mean())
        else:
            continue

In [30]:
# All missing values were imputed
print("TRAINING FEATURES MISSING:")
print(pd.isnull(x_train).sum())
print("")
print("TEST FEATURES MISSING:")
print(pd.isnull(x_test).sum())

TRAINING FEATURES MISSING:
dmspols_2011            0
l7_2011_1               0
l7_2011_2               0
l7_2011_3               0
l7_2011_4               0
l7_2011_5               0
l7_2011_6               0
l7_2011_7               0
viirs_2012              0
uid                     0
dmspols_2011_imputed    0
viirs_2012_imputed      0
dtype: int64

TEST FEATURES MISSING:
dmspols_2011            0
l7_2011_1               0
l7_2011_2               0
l7_2011_3               0
l7_2011_4               0
l7_2011_5               0
l7_2011_6               0
l7_2011_7               0
viirs_2012              0
uid                     0
dmspols_2011_imputed    0
viirs_2012_imputed      0
dtype: int64


## 4. Feature Generation

[Landsat 7 specs](https://landsat.usgs.gov/sites/default/files/documents/si_product_guide.pdf#page=14)

Create indices from every possible pair of Landsat 7 band.
- Normalized Difference Vegetation Index, NDVI = $\frac{NIR - Red}{NIR + Red}$ is formed from the (NIR, Red) pair.
- Normalized Difference Built-up Index, NDBI = $\frac{SWIR1 - NIR}{SWIR1 + NIR}$ is formed from the (NIR, SWIR1) pair.
- Normalized Difference Water Index, NDWO = $\frac{NIR - SWIR1}{NIR + SWIR1}$ is also formed from the (NIR, SWIR1) pair.
- Modified NDWI, MNDWI = $\frac{Green - SWIR1}{Green + SWIR1}$ is formed from the (NIR, Green) pair. And so on.


| Band | 1 | 2 | 3 | 4 | 5 | 6 | 7
| ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- 
| 1 | NA 
| 2 | ? | NA 
| 3 | ? | ? | NA 
| 4 | ? | ? | NDVI | NA
| 5 | ? | MNDWI | ? | NDBI, NDWI | NA 
| 6 | ? | ? | ? | ? | ? | NA 
| 7 | ? | ? | ? | ? | ? | ? | NA



In [31]:
# Create ratios 
# Note that ratio of Band A to Band B is the same as ratio of Band B to Band A
# Solution: only create ratios where A < B
for df in (x_train, x_test):
    for i in range(1, 8):
        for j in range(1, 8):

            if i >= j:
                continue
            else:
                band1 = f'l7_2011_{i}'
                band2 = f'l7_2011_{j}'
                new_var = f'ratio_{i}_{j}'
                df[new_var] = abs((df[band1] - df[band2]) / (df[band1] + df[band2]))

In [32]:
x_train.head()

,dmspols_2011,l7_2011_1,l7_2011_2,l7_2011_3,l7_2011_4,l7_2011_5,l7_2011_6,l7_2011_7,viirs_2012,uid,dmspols_2011_imputed,viirs_2012_imputed,ratio_1_2,ratio_1_3,ratio_1_4,ratio_1_5,ratio_1_6,ratio_1_7,ratio_2_3,ratio_2_4,ratio_2_5,ratio_2_6,ratio_2_7,ratio_3_4,ratio_3_5,ratio_3_6,ratio_3_7,ratio_4_5,ratio_4_6,ratio_4_7,ratio_5_6,ratio_5_7,ratio_6_7
374,24.0,901.910885,1276.550363,1411.094775,2996.116401,2546.972714,3031.464877,1907.712192,1.229348,4900320,0,0,0.171974,0.220139,0.537248,0.476984,0.541406,0.357984,0.050060,0.402457,0.332265,0.407360,0.198213,0.359643,0.286978,0.364738,0.149637,0.081028,0.005864,0.221950,0.086851,0.143503,0.227518
2388,31.0,1010.649942,1344.685760,1589.681990,2445.176189,2589.602233,3018.334252,2046.523202,4.102749,25703397,0,0,0.141821,0.222676,0.415104,0.438567,0.498310,0.338834,0.083492,0.290377,0.316427,0.383599,0.206958,0.212026,0.239256,0.310036,0.125637,0.028686,0.104907,0.088753,0.076451,0.117141,0.191873
3461,6.5,1016.671152,1320.956019,1479.582899,2359.837384,2234.348958,3026.724826,1685.984520,0.301120,37700984,0,0,0.130168,0.185443,0.397797,0.374553,0.497120,0.247650,0.056642,0.282244,0.256910,0.392340,0.121395,0.229268,0.203226,0.343328,0.065202,0.027315,0.123806,0.166555,0.150611,0.139877,0.284495
3662,0.0,617.041618,787.066415,809.077871,2105.904002,1763.348318,2938.720708,1204.052204,0.109788,39800512,0,0,0.121091,0.134656,0.546784,0.481563,0.652934,0.322340,0.013790,0.455877,0.382793,0.577503,0.209423,0.444883,0.370961,0.568238,0.196199,0.088533,0.165090,0.272466,0.249969,0.188480,0.418722
83,18.0,880.414847,1152.104803,1318.878894,2572.979330,2284.983406,3030.307132,1783.694032,1.105853,1702700,0,0,0.133672,0.199366,0.490116,0.443726,0.549743,0.339055,0.067493,0.381434,0.329604,0.449072,0.215134,0.322237,0.268075,0.393505,0.149816,0.059283,0.081618,0.181167,0.140223,0.123207,0.258956


In [33]:
x_test.head()

,dmspols_2011,l7_2011_1,l7_2011_2,l7_2011_3,l7_2011_4,l7_2011_5,l7_2011_6,l7_2011_7,viirs_2012,uid,dmspols_2011_imputed,viirs_2012_imputed,ratio_1_2,ratio_1_3,ratio_1_4,ratio_1_5,ratio_1_6,ratio_1_7,ratio_2_3,ratio_2_4,ratio_2_5,ratio_2_6,ratio_2_7,ratio_3_4,ratio_3_5,ratio_3_6,ratio_3_7,ratio_4_5,ratio_4_6,ratio_4_7,ratio_5_6,ratio_5_7,ratio_6_7
3145,6.000000,1067.911645,1347.568656,1496.182648,2448.161935,2328.447567,3004.758980,1737.813586,0.337368,34902063,0,0,0.115777,0.167026,0.392554,0.371143,0.475572,0.238762,0.052260,0.289956,0.266832,0.380760,0.126482,0.241353,0.217607,0.335169,0.074716,0.025063,0.102073,0.169697,0.126811,0.145252,0.267143
3888,0.000000,909.066599,1054.835316,1108.651480,1493.869704,1341.081399,2903.133198,1080.304266,0.173900,43301419,0,0,0.074224,0.098916,0.243370,0.191994,0.523075,0.086076,0.024875,0.172258,0.119472,0.466981,0.011928,0.148017,0.094880,0.447303,0.012950,0.053895,0.320505,0.160659,0.368043,0.107697,0.457602
1117,7.666667,843.911082,1138.947636,1333.943429,2591.361184,2285.129243,3031.391065,1782.293734,0.276674,14502621,0,0,0.148794,0.225007,0.508679,0.460594,0.564467,0.357315,0.078853,0.389355,0.334742,0.453787,0.220230,0.320336,0.262826,0.388847,0.143876,0.062798,0.078259,0.184987,0.140367,0.123625,0.259489
3348,10.000000,1188.818870,1759.164822,2251.561736,3211.930984,3842.226267,3058.202097,3118.564065,0.925885,36901045,0,0,0.193470,0.308903,0.459720,0.527407,0.440163,0.448009,0.122770,0.292243,0.371883,0.269657,0.278695,0.175779,0.261031,0.151916,0.161449,0.089351,0.024518,0.014749,0.113620,0.103963,0.009772
5245,2.000000,1543.693146,2214.106303,2833.584665,3405.777374,4622.612692,3047.386146,3561.186030,0.158197,61200110,0,0,0.178406,0.294679,0.376219,0.499313,0.327525,0.395209,0.122725,0.212046,0.352290,0.158373,0.233249,0.091707,0.239938,0.036355,0.113781,0.151567,0.055537,0.022307,0.205375,0.129699,0.077747


In [34]:
# check that lengths match
print(len(x_train) == len(y_train))
print(len(x_test) == len(y_test))

True
True


### 4.1 Define feature groups

1. Daytime-only: Landsat 7 band data and computed indices
2. Nighttime-only: DMSP and VIIRS data + imputed flags
3. All features

In [35]:
DAY_FEATURES = df.filter(regex='l7|ratio', axis=1).columns.tolist()
NIGHT_FEATURES = ['dmspols_2011', 'viirs_2012', 'dmspols_2011_imputed', 'viirs_2012_imputed']
ALL_FEATURES = df.columns.tolist()

print("Day-only:", DAY_FEATURES)
print("-----")
print("Night-only:", NIGHT_FEATURES)

Day-only: ['l7_2011_1', 'l7_2011_2', 'l7_2011_3', 'l7_2011_4', 'l7_2011_5', 'l7_2011_6', 'l7_2011_7', 'ratio_1_2', 'ratio_1_3', 'ratio_1_4', 'ratio_1_5', 'ratio_1_6', 'ratio_1_7', 'ratio_2_3', 'ratio_2_4', 'ratio_2_5', 'ratio_2_6', 'ratio_2_7', 'ratio_3_4', 'ratio_3_5', 'ratio_3_6', 'ratio_3_7', 'ratio_4_5', 'ratio_4_6', 'ratio_4_7', 'ratio_5_6', 'ratio_5_7', 'ratio_6_7']
-----
Night-only: ['dmspols_2011', 'viirs_2012', 'dmspols_2011_imputed', 'viirs_2012_imputed']


### 4.2 Pickle cleaned data for future use

In [36]:
clean_data = [x_train, x_test, y_train, y_test]

output_path = os.path.join('output', 'final_data.pkl')
with open(output_path, 'wb') as f:
    pickle.dump(obj=clean_data,
                file=f,
                protocol=pickle.HIGHEST_PROTOCOL)

## 5. Train and Evaluate Regressors

### 5.1 Training

In [37]:
x_all = x_test.append(x_train)
y_all = y_test.append(y_train)

In [38]:
# Define a TrainedRegressor object to hold key results information
class TrainedRegressor:
    
    def __init__(self, method, params, features, regressor):
        self.method = method
        self.params = params
        self.regressor = regressor
        self.features = features
    
    def __repr__(self):
        return f'Trained {self.method} on feature set {self.features} with params {self.params}'

In [39]:
# Use GRID_MAIN for full grid search
# parameters = cf.GRID_TEST_CLASS
parameters = GRID_TEST_CLASS

results_df = pd.DataFrame()
results_df_all = pd.DataFrame()
results_df_trainedonly_all = pd.DataFrame()

x_trainedonly_all = x_all.copy()

trained_list = []
trained_list_all = []
count = 0
# print('Training model ', end='')
for i in parameters['regressors']:
    for j in parameters[i]:
        for k in ('DAY_FEATURES', 'NIGHT_FEATURES', 'ALL_FEATURES'):
        
            print(f'Model {count}: Training {i} on {k} with params {str(j)}')

            # A. Train ------------------------------------
            # Initialize regressor, fit data, then append TrainedRegressor object to list
            # 1. Train Data
            regressor = eval(i)(**j)
            trained = regressor.fit(x_train[eval(k)], y_train)
            trained_list.append(TrainedRegressor(i, str(j), k, trained))

            # 2. All Data
            trained_all = regressor.fit(x_all[eval(k)], y_all)
            trained_list_all.append(TrainedRegressor(i, str(j), k, trained_all))
            
            
            
            
            
            
            
            # B. Results -------------------------------------
            # 1. Trained Model on Test Data - - - - - - - - - -
            pred_labels = trained_list[count].regressor.predict(x_test[eval(k)])

            pred_dict = {
                'regressor': trained_list[count].method,
                'features': trained_list[count].features,
                'params': trained_list[count].params,
                'accuracy_score': accuracy_score(y_true=y_test, y_pred=pred_labels)        
            }
    
            results_df = results_df.append(pred_dict, ignore_index=True) \
                .sort_values(by='accuracy_score', ascending=False, axis=0) \
                [['regressor', 'params', 'features', 'accuracy_score']]
        
            results_df.to_csv("/Users/robmarty/Desktop/pov_results.csv")
            
            x_test['y_true'] = y_test
            x_test['y_predict_' + str(count)] = pred_labels
            x_test.to_csv(os.path.join(final_data_file_path, 'Data with Predicted Income', 'pov_opm_data_with_predictions_traineddatamodel_testdatapredict.csv'))
            
            
            
            
            
            
            
            # 2. Trained All Model on All Data - - - - - - - - - -
            pred_labels_all = trained_list_all[count].regressor.predict(x_all[eval(k)])

            # Append results to dataframe and sort by R^2
            pred_dict = {
                'regressor': trained_list_all[count].method,
                'features': trained_list_all[count].features,
                'params': trained_list_all[count].params,
                'accuracy_score': accuracy_score(y_true=y_all, y_pred=pred_labels_all)        
            }
    
            results_df_all = results_df_all.append(pred_dict, ignore_index=True) \
                .sort_values(by='accuracy_score', ascending=False, axis=0) \
                [['regressor', 'params', 'features', 'accuracy_score']]
        
            results_df_all.to_csv("/Users/robmarty/Desktop/pov_results_all.csv")

            # ALL
            x_trainedonly_all['y_true'] = y_all
            x_trainedonly_all['y_predict_' + str(count)] = trained_list_all[count].regressor.predict(x_all[eval(k)])
            x_trainedonly_all.to_csv(os.path.join(final_data_file_path, 'Data with Predicted Income', 'pov_opm_data_with_predictions_alldatamodel_alldatapredict.csv'))
            
            
            
            
            
            
            # 3. Trained Model on All Data - - - - - - - - - -
            pred_labels_trainedonly_all = trained_list[count].regressor.predict(x_all[eval(k)])

            # Append results to dataframe and sort by R^2
            pred_dict = {
                'regressor': trained_list[count].method,
                'features': trained_list[count].features,
                'params': trained_list[count].params,
                'accuracy_score': accuracy_score(y_true=y_all, y_pred=pred_labels_trainedonly_all)        
            }
    
            results_df_trainedonly_all = results_df_trainedonly_all.append(pred_dict, ignore_index=True) \
                .sort_values(by='accuracy_score', ascending=False, axis=0) \
                [['regressor', 'params', 'features', 'accuracy_score']]
        
            results_df_trainedonly_all.to_csv("/Users/robmarty/Desktop/pov_results_trainedonly_all.csv")

            # ALL
            x_all['y_true'] = y_all
            x_all['y_predict_' + str(count)] = trained_list[count].regressor.predict(x_all[eval(k)])
            x_all.to_csv(os.path.join(final_data_file_path, 'Data with Predicted Income', 'pov_opm_data_with_predictions_testdatamodel_alldatapredict.csv'))

            ####
            count += 1


Model 0: Training LinearSVC on DAY_FEATURES with params {'penalty': 'l2', 'C': 0.01, 'loss': 'epsilon_insensitive', 'max_iter': 1000.0, 'random_state': 0}
Model 1: Training LinearSVC on NIGHT_FEATURES with params {'penalty': 'l2', 'C': 0.01, 'loss': 'epsilon_insensitive', 'max_iter': 1000.0, 'random_state': 0}
Model 2: Training LinearSVC on ALL_FEATURES with params {'penalty': 'l2', 'C': 0.01, 'loss': 'epsilon_insensitive', 'max_iter': 1000.0, 'random_state': 0}
Model 3: Training LinearSVC on DAY_FEATURES with params {'penalty': 'l2', 'C': 0.01, 'loss': 'epsilon_insensitive', 'max_iter': 100000.0, 'random_state': 0}
Model 4: Training LinearSVC on NIGHT_FEATURES with params {'penalty': 'l2', 'C': 0.01, 'loss': 'epsilon_insensitive', 'max_iter': 100000.0, 'random_state': 0}
Model 5: Training LinearSVC on ALL_FEATURES with params {'penalty': 'l2', 'C': 0.01, 'loss': 'epsilon_insensitive', 'max_iter': 100000.0, 'random_state': 0}
Model 6: Training LinearSVC on DAY_FEATURES with params {'pe

In [ ]:
#y_test
#pred_labels
#parameters

### 5.2 Prediction and Evaluation

In [ ]:
results_df = pd.DataFrame()
for i in trained_list:
    
    # Get predicted results from test data
    features = eval(i.features)
    pred_labels = i.regressor.predict(x_test[features])
    
    # Append results to dataframe and sort by R^2
    pred_dict = {
        'regressor': i.method,
        'features': i.features,
        'params': i.params,
        'r2': r2_score(y_true=y_test, y_pred=pred_labels)        
    }
    
    results_df = results_df.append(pred_dict, ignore_index=True) \
        .sort_values(by='r2', ascending=False, axis=0) \
        [['regressor', 'params', 'features', 'r2']]

results_df.shape

In [ ]:
results_df